In [ ]:
import pandas as pd
from prophet import Prophet

# Sample data creation (replace with your actual data)
# Assuming daily data
data = pd.DataFrame({
    'ds': pd.date_range(start='2023-01-01', periods=500, freq='D'),
    'y': np.random.randn(500).cumsum(),  # Replace with your target variable
    'structure': np.random.randn(500),
    'pattern': np.random.randn(500),
    'trigger': np.random.randn(500),
    'double': np.random.randn(500)
})

# Initialize Prophet
model = Prophet()

# Add additional regressors
model.add_regressor('structure')
model.add_regressor('pattern')
model.add_regressor('trigger')
model.add_regressor('double')

# Fit the model
model.fit(data)

# Create future dataframe
future = model.make_future_dataframe(periods=30)

# For the future regressors, you need to provide the values.
# Here, we'll use the last known values or some projection.
# Replace these with your actual future regressor values.
future['structure'] = np.nan
future['pattern'] = np.nan
future['trigger'] = np.nan
future['double'] = np.nan

# Fill NaNs with the last known value or any other strategy
future[['structure', 'pattern', 'trigger', 'double']] = data[['structure', 'pattern', 'trigger', 'double']].iloc[-1].values

# Alternatively, you can project future regressors based on your domain knowledge

# Make predictions
forecast = model.predict(future)

# Plot the forecast
fig1 = model.plot(forecast)
fig2 = model.plot_components(forecast)


In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

# 1. Prepare Your Data
# Replace this with your actual data
# Assuming daily data with 'ds' as date and 'y' as target variable
data = pd.DataFrame({
    'ds': pd.date_range(start='2023-01-01', periods=1000, freq='D'),
    'y': np.sin(np.linspace(0, 50, 1000)) + np.random.normal(0, 0.5, 1000)  # Example target
})

# Additional features (replace with your actual features)
# For illustration, we'll create some synthetic features
data['feature1'] = np.random.randn(1000)
data['feature2'] = np.random.randn(1000)
data['feature3'] = np.random.randn(1000)

# 2. Fit Prophet Model
prophet_model = Prophet()
prophet_model.fit(data[['ds', 'y']])

# 3. Make In-Sample Predictions and Calculate Residuals
in_sample_forecast = prophet_model.predict(data[['ds']])
data['y_pred_prophet'] = in_sample_forecast['yhat']
data['residual'] = data['y'] - data['y_pred_prophet']

# 4. Prepare Data for CNN
# Define window size
window_size = 30  # Number of past days to consider

# Function to create CNN input
def create_cnn_input(data, window_size):
    X = []
    y = []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:i+window_size][['feature1', 'feature2', 'feature3']].values)
        y.append(data.iloc[i+window_size]['residual'])
    return np.array(X), np.array(y)

X, y = create_cnn_input(data, window_size)

# Split into training and testing
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 5. Build and Train CNN Model
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(window_size, 3)),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)
])

cnn_model.compile(optimizer='adam', loss='mse')
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

cnn_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop]
)

# 6. Make Predictions
# Prepare test input
X_future, _ = create_cnn_input(data.iloc[-(window_size + len(X_test)):], window_size)

# Predict residuals
residual_pred = cnn_model.predict(X_test).flatten()

# 7. Combine Prophet and CNN Predictions
# Forecast horizon
forecast_horizon = len(X_test)

# Get Prophet's forecast for the test period
test_dates = data['ds'].iloc[window_size:split+window_size].reset_index(drop=True)
prophet_test_forecast = prophet_model.predict(test_dates.to_frame())

# Final Forecast
final_forecast = prophet_test_forecast['yhat'].values + residual_pred

# 8. Evaluate the Model
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, final_forecast)
print(f'Mean Squared Error: {mse}')

# 9. Visualize the Results
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(test_dates, y_test, label='Actual')
plt.plot(test_dates, final_forecast, label='Combined Forecast', alpha=0.7)
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()
